In [1]:
from fake_headers import Headers
import requests
import hashlib
import re
import pandas as pd
import scrapy
from crochet import setup, wait_for
from scrapy.crawler import CrawlerRunner
import argparse
import io
import imagehash
from PIL import Image

PATH_TO_CLASSES = "./data/classes.csv"
PATH_TO_ALL_HASH = "./data/allhash.csv"
classes = pd.read_csv(PATH_TO_CLASSES)
model_request = []
model_name = []

# my_cookies = {'dp1':r'',
#            'ns1':r'',
#            's':r'',
#            'JSESSIONID':r'',
#            'ebay':r'',
#            'nonsession':r''
#            },

# fill cookies with yours from ebay.com, set up United Kindom location before.

for index, row in classes.iterrows():
    if classes.at[index, "brand"] in classes.at[index, "model"]:
        result = classes.at[index, "model"]
    else:
        result = classes.at[index, "brand"] + " " + classes.at[index, "model"]
    url = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=4&_sacat=11450&_sop=12&_ipg=240'
    url1 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=4&_sacat=11450&_sop=12&_ipg=240&_pgn=2'
    url2 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=4&_sacat=11450&_sop=12&_ipg=240&_pgn=3'
    url3 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=1&_sacat=11450&_sop=12&_ipg=240&_pgn=4'
    
    # after first one finished, uncoment this and run second version 
    # url = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240'
    # url1 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&_pgn=2'
    # url2 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&_pgn=3'
    # url3 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&_pgn=4' 
    
    # after secone one finished, uncoment this and run third version 
    # url = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&LH_Complete=1&LH_Sold=1&_ipg=240'
    # url1 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&LH_Complete=1&LH_Sold=1&_pgn=2'
    # url2 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&LH_Complete=1&LH_Sold=1&_pgn=3'
    # url3 = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={"+".join(result.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=240&LH_Complete=1&LH_Sold=1&_pgn=4' 

    model_request.append(url)
    model_request.append(url1)
    model_request.append(url2)
    model_request.append(url3)

    model_name.append(result)
    model_name.append(result)
    model_name.append(result)
    model_name.append(result)
    

print(len(model_request))
setup()
seen_sites = set()


main_df = pd.read_csv(PATH_TO_ALL_HASH) # Dataframe with test data with column avg hash
main_df['class_id'] = -1
class_df = pd.read_csv(PATH_TO_CLASSES) # Dataframe with column class_id and model
    

class EbaySpider(scrapy.Spider):
    """
    Our spider for parse image from ebay and analyze avg_hash them, therefore
    fill dataframe finded class_id.
    """
    name = "Serega"
    custom_settings = {
        'CONCURRENT_REQUESTS': 32,
    }
    
    # 1 stage: iterate on search url
    def start_requests(self):
        meta = zip(model_request[:1000], model_name[:1000])
        for url, model in meta:
            yield scrapy.Request(url=url, 
                                 callback=self.parse,
                                #  cookies=my_cookies, # uncomment this after all first version of the run finished
                                 cb_kwargs=dict(find_model=model))

    # 2 stage: find page url boots and iterate
    def parse(self, response, find_model):
        LIMIT_ITER = 240 # num of urls we iterate in loops

        links = response.xpath(r'//div[@class = "srp-river-results clearfix"]//@href').getall()
        for quote in links[:LIMIT_ITER]: 
            yield scrapy.Request(url=quote + "", 
                                 callback=self.parse_hash,
                                #  cookies=my_cookies,
                                 cb_kwargs=dict(find_model=find_model))
    
    # 3 stage: iterate on image in page boots and analyze them average hash
    def parse_hash(self, response, find_model):
        global class_df
        global main_df
        tmp_model = response.xpath(r'//div[@class="ux-layout-section ux-layout-section--features"]//span[@itemprop="model"]//span[@class="ux-textspans"]/text()').get()
        tmp_model = tmp_model.lower()
        if tmp_model != find_model: # new balance 57/40  ||| 57/40
            if len(class_df[class_df.model == tmp_model]) > 0:
                fin = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l500\.jpg", str(response.text))
                fin1 = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l1600\.jpg", str(response.text))
                fin2 = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l640\.jpg", str(response.text))
                fin = (list(set(fin)))
                fin1 = (list(set(fin1)))
                fin2 = (list(set(fin2)))
                fin.extend(list(set(fin1)))
                fin.extend(list(set(fin2)))

                session = requests.Session()
                for img_url in list(set(fin)):
                    try:
                        img_data = session.get(img_url, timeout=20)
                    except:
                        headers = Headers(headers=True).generate()
                        session.headers = headers
                        img_data = session.get(img_url, timeout=20)


                    img_data = img_data.content
                    with Image.open(io.BytesIO(img_data)) as img:
                        avg_hash = str(imagehash.average_hash(img, 24))
                    
                    main_df.loc[(main_df['avg'] == avg_hash), 'class_id'] = self.class_df[self.class_df.model == tmp_model].id.values[0]
            
            else:
                pass
        else:
            print(tmp_model)
            fin = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l500\.jpg", str(response.text))
            fin1 = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l1600\.jpg", str(response.text))
            fin2 = re.findall(r"https:\/\/i\.ebayimg\.com\/(?!thumbs\/)images\/g\/[a-zA-Z0-9]*\/s-l640\.jpg", str(response.text))

            fin = (list(set(fin)))
            fin1 = (list(set(fin1)))
            fin2 = (list(set(fin2)))
            fin.extend(list(set(fin1)))
            fin.extend(list(set(fin2)))
            session = requests.Session()
            for img_url in list(set(fin)):
                try:
                    img_data = session.get(img_url, timeout=20)
                except:
                    headers = Headers(headers=True).generate()
                    session.headers = headers
                    img_data = session.get(img_url, timeout=20)

                img_data = img_data.content
                with Image.open(io.BytesIO(img_data)) as img:
                    avg_hash = str(imagehash.average_hash(img, 24))
                main_df.loc[(main_df['avg'] == avg_hash), 'class_id'] = class_df[class_df.model == tmp_model].id.values[0]
        
# @wait_for(10)

def run_spider():
    """run spider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(EbaySpider)
    return d


run_spider()

13188
https://www.ebay.com/sch/i.html?_from=R40&_nkw=new+balance+57/40&_in_kw=4&_sacat=11450&_sop=12&_ipg=240


new balance 57/40
new balance fresh foam roav
new balance fresh foam roav
new balance 327
new balance 574
new balance fresh foam roav
new balance 1080
new balance 1080
new balance 1080
new balance 1080
new balance 1080
new balance 1080
new balance 5740
new balance 1080
new balance 1080
new balance 1080
new balance 1080
new balance 1080
new balance 1080


In [32]:
import pandas as pd
main_df.class_id.value_counts()


-1      84562
 5        167
 273      151
 4        114
 237      107
        ...  
 124        2
 196        1
 288        1
 159        1
 245        1
Name: class_id, Length: 112, dtype: int64

In [24]:
main_df.to_csv("one_of_the_possible_fill.csv",index=False)